In [ ]:
%load_ext autoreload
%autoreload 3

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os

try:
    import pudl
    from pudl.etl import defs
    from pudl.analysis.timeseries_evaluation import plot_imputation, plot_correlation, plot_compare_imputation
    from dagster import AssetKey
    logger = pudl.logging_helpers.get_logger("pudl")
except ImportError:
    print("PUDL not installed in Python environment")
    pass

## Visualization Settings

In [ ]:
%matplotlib inline

In [ ]:
matplotlib.rcParams["figure.figsize"] = (10, 6)
matplotlib.rcParams["figure.dpi"] = 150
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_colwidth", 1000)

# Nice dark theme for Matplotlib... but only if you have matplotx installed.
try:
    import matplotx
    matplotlib.style.use(matplotx.styles.onedark)
except ImportError:
    pass

# Data access shortcuts

In [ ]:
try:
    import pudl
    def get_parquet(table: str) -> pd.DataFrame:
        """Read data from local PUDL Parquet outputs."""
        return pd.read_parquet(Path(os.environ["PUDL_OUTPUT"]) / f"parquet/{table}.parquet")

    def get_asset(table: str) -> pd.DataFrame:
        """Read data from locally persisted Dagster assets."""
        return defs.load_asset_value(AssetKey(table))
except ImportError:
    print("PUDL not installed in Python environment")
    pass

def get_s3_tmp(table: str) -> pd.DataFrame:
    """Read data from our AWS open data registry S3 bucket."""
    return pd.read_parquet(f"s3://pudl.catalyst.coop/tmp/eia930/{table}.parquet")

def get_s3_nightly(table: str) -> pd.DataFrame:
    """Read data from our AWS open data registry S3 bucket."""
    return pd.read_parquet(f"s3://pudl.catalyst.coop/nightly/{table}.parquet")


## Read in remote data

In [ ]:
eia930_sub = get_asset("out_eia930__hourly_subregion_demand")
eia930_ops = get_asset("core_eia930__hourly_operations")
eia930_ba = get_asset("out_eia930__hourly_operations")

In [ ]:
eia930_sub.info(show_counts=True)

In [ ]:
eia930_sub.loc[
    (eia930_sub.demand_imputed_pudl_mwh_imputation_code.isna())
    & (eia930_sub.demand_imputed_pudl_mwh.notna())
]

In [ ]:
imputation_codes = get_asset("core_pudl__codes_imputation_reasons")
imputation_codes

# Find some imputed data

Calculate the proportion of imputed values by subregion to identify areas with a lot of imputation happening so we can see what the results look like.

In [ ]:
bad_data = (
    eia930_sub.groupby(
        [
            "balancing_authority_code_eia",
            "balancing_authority_subregion_code_eia",
            eia930_sub["datetime_utc"].dt.year  # Extract the year from datetime_utc
        ], observed=True)
    ["demand_imputed_pudl_mwh_imputation_code"]
    .apply(lambda x: x.notnull().mean()).sort_values(ascending=False)
)
bad_data.head(50).tail(25)

In [ ]:
idx_cols = ["balancing_authority_code_eia", "balancing_authority_subregion_code_eia"]
reported_col = "demand_reported_mwh"
imputed_col = "demand_imputed_pudl_mwh"

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("SWPP", "OPPD"),
    start_date="2019-11-01",
    end_date="2019-12-31",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("CISO", "PGAE"),
    start_date="2019-02-01",
    end_date="2019-02-20",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("CISO", "VEA"),
    start_date="2019-02-01",
    end_date="2019-02-20",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("SWPP", "INDN"),
    start_date="2019-12-01",
    end_date="2019-12-31",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("SWPP", "INDN"),
    start_date="2024-12-01",
    end_date="2024-12-31",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("PNM", "KCEC"),
    start_date="2022-06-15",
    end_date="2022-07-15",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
plot_imputation(
    eia930_sub,
    idx_cols=idx_cols,
    idx_vals=("CISO", "VEA"),
    start_date="2019-12-01",
    end_date="2019-12-31",
    reported_col=reported_col,
    imputed_col=imputed_col,
)


## Repeat with BA data

In [ ]:
bad_data = (
    eia930_ba.groupby(
        [
            "balancing_authority_code_eia",
            eia930_ba["datetime_utc"].dt.year  # Extract the year from datetime_utc
        ], observed=True)
    ["demand_imputed_pudl_mwh_imputation_code"]
    .apply(lambda x: x.notnull().mean()).sort_values(ascending=False)
)
bad_data.head(50).tail(25)

In [ ]:
idx_cols = ["balancing_authority_code_eia"]
plot_imputation(
    eia930_ba,
    idx_cols=idx_cols,
    idx_vals=("SC",),
    start_date="2017-12-01",
    end_date="2017-12-31",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

plot_imputation(
    eia930_ba,
    idx_cols=idx_cols,
    idx_vals=("SEC",),
    start_date="2017-11-01",
    end_date="2017-12-15",
    reported_col=reported_col,
    imputed_col=imputed_col,
)
plot_imputation(
    eia930_ba,
    idx_cols=idx_cols,
    idx_vals=("GVL",),
    start_date="2019-02-01",
    end_date="2019-03-01",
    reported_col=reported_col,
    imputed_col=imputed_col,
)
plot_imputation(
    eia930_ba,
    idx_cols=idx_cols,
    idx_vals=("BANC",),
    start_date="2019-04-01",
    end_date="2019-05-01",
    reported_col=reported_col,
    imputed_col=imputed_col,
)

In [ ]:
assert False

In [ ]:
eia930_ba

# Is subregion demand data consistent with BA-level reporting?

- Investigate how consistent the more granular subregion data is with the BA-level data.
- Sum up all reported demand for each of them and look at the proportional difference.
- Plot the correlation between the aggregated subregion data and the BA-level data.
- Look more closely at some individual spans of time where the differences are significant.

In [ ]:
eia930_bas = (
    eia930_ops
    .loc[:, ["balancing_authority_code_eia", "datetime_utc", "demand_reported_mwh"]]
    .set_index(["balancing_authority_code_eia", "datetime_utc"])
)

In [ ]:
eia930_sub_agg = (
    eia930_sub.groupby(["balancing_authority_code_eia", "datetime_utc"])[
        "demand_reported_mwh"
    ].sum().to_frame()
)

In [ ]:
eia930_both = pd.merge(
    eia930_bas,
    eia930_sub_agg,
    left_index=True,
    right_index=True,
    how="inner",
    suffixes=("_ba", "_sub")
)

## Bulk proportions

Note that our BA-level data has not yet been imputed, so this isn't a perfect compairson.

Differences as (BA-sum(subregions))/BA (from Alicia Wongel)

| Balancing Authority | Difference (%) |
|---------------------|----------------|
| CISO               | 0.005%         |
| ERCO               | -0.042%        |
| ISNE               | 0.060%         |
| MISO               | 0.035%         |
| PJM                | 1.604%         |
| SWPP               | 0.022%         |
| NYIS               | 0.031%         |

Looking below, in general the bulk demand is only off by a fraction of a percent, with the exception of ERCOT (5% off) and PJM (18% off) -- but again the BA level data here still has all of its outliers and weirdness.

In [ ]:
ba_totals = eia930_both.groupby("balancing_authority_code_eia").sum()
ba_totals["pct_diff"] = (ba_totals["demand_reported_mwh_ba"] - ba_totals["demand_reported_mwh_sub"]) / ba_totals["demand_reported_mwh_ba"]
ba_totals

## Correlation plot by BA

- Many differences between these two time series could be washed out by the aggregation.
- Plotting the individual data points will let us see whether there are any systematic differences between the two.
- This obscures the time dimension, but is still useful for a quick check.
- Interestingly, even though the bulk difference above for ISNE was less than 1%, it seems to have more scatter than the other BAs.
- However, the scatter seems to be more or less evenly distributed above and below the line of 1:1 correlation, so it's cancelling out.
- PJM has less scatter, but it's all below the line, leading to a bigger overall bulk difference.

In [ ]:
all_bas = list(eia930_both.index.get_level_values(0).unique())

plot_correlation(
    eia930_both.reset_index(),
    idx_cols=["balancing_authority_code_eia"],
    idx_vals=all_bas,
    timeseries_x="demand_reported_mwh_ba",
    timeseries_y="demand_reported_mwh_sub",
    xlabel="BA Reported Demand [MWh]",
    ylabel="Aggregated Subregion Demand [MWh]",
    title="Correlation between BA Reported Demand and Aggregated Subregion Demand",
    xylim=(1e3, 2e5),
    alpha=0.1,
)

## Identify months with large discrepancies
- Which individual periods of time are most responsible for the differences between the BA-level and subregion-level data?
- A handful of months seem to just have missing (zero) subregional data.
- However, ISNE doesn't seem to show up much in the top fractional differences on a monthly basis.

In [ ]:
eia930_bymonth = eia930_both.reset_index()
eia930_bymonth = eia930_bymonth.groupby(
    [
        "balancing_authority_code_eia",
        eia930_bymonth["datetime_utc"].dt.to_period("M")
    ]
)[["demand_reported_mwh_ba", "demand_reported_mwh_sub"]].sum()
eia930_bymonth["frac_diff"] = (
    eia930_bymonth["demand_reported_mwh_ba"] - eia930_bymonth["demand_reported_mwh_sub"]
) / eia930_bymonth["demand_reported_mwh_ba"]
eia930_bymonth.sort_values("frac_diff", ascending=False).head(25)

In [ ]:
eia930_bymonth.sort_values("frac_diff", ascending=False).loc["ISNE"].head(25)

In [ ]:
isne_ba = eia930_both.sort_index().loc[("ISNE", slice("2019-02-01","2019-02-28")), "demand_reported_mwh_ba"].reset_index()
isne_sub = eia930_both.sort_index().loc[("ISNE", slice("2019-02-01","2019-02-28")), "demand_reported_mwh_sub"].reset_index()
plt.plot(isne_ba["datetime_utc"], isne_ba["demand_reported_mwh_ba"], label="BA Reported", lw=1)
plt.plot(isne_sub["datetime_utc"], isne_sub["demand_reported_mwh_sub"], label="Subregion Reported", lw=1)
plt.title("ISNE BA vs. Subregion Demand")
plt.ylabel("Demand [MWh]")
plt.legend()

## Compare old and new FERC-714 Imputations

- This currently takes a long time to run, so it's below the assert False

In [ ]:
# For pulling data from S3:
new_imputed = get_asset("out_ferc714__hourly_planning_area_demand")
old_imputed = pd.read_parquet("_out_ferc714__hourly_imputed_demand.parquet")

# For running locally w/ PUDL environment.
#new_imputed = get_asset("out_ferc714__hourly_planning_area_demand")
#old_imputed = pd.read_parquet("_out_ferc714__hourly_imputed_demand.parquet")

In [ ]:
new_imputed.info()

In [ ]:
ferc714_both = pd.merge(
    left=new_imputed.loc[:, ["respondent_id_ferc714", "datetime_utc", "demand_imputed_pudl_mwh"]],
    right=old_imputed.loc[:, ["respondent_id_ferc714", "datetime_utc", "demand_mwh"]],
    on=["respondent_id_ferc714", "datetime_utc"],
    how="outer",
).rename(
    columns={
        "demand_imputed_pudl_mwh": "demand_mwh_new",
        "demand_mwh": "demand_mwh_old",
    }
)
ferc714_both.info()

In [ ]:
all_rids = list(ferc714_both["respondent_id_ferc714"].unique())

plot_correlation(
    ferc714_both,
    idx_cols=["respondent_id_ferc714"],
    idx_vals=all_rids,
    timeseries_x="demand_mwh_old",
    timeseries_y="demand_mwh_new",
    xlabel="Old Imputed Demand [MWh]",
    ylabel="New Imputed Demand [MWh]",
    title="Correlation between Old and New FERC-714 Imputed Planning Area Demand",
    xylim=(1, 2e5),
    legend=False,
    alpha=0.2,
)